In [1]:
import sys
import pandas as pd
sys.path.append("C:/Users/ping/MyDrive/py_files/python/py379/")
from myUtils import pickle_load, pickle_dump
path_pickle_dump = 'C:/Users/ping/OneDrive/Documents/jenn_bb_sales/'
pd.set_option('max_colwidth', 12, 'display.max_columns', 18, 'display.width', 1200, 'display.max_rows',100)

In [ ]:
# # read Square csv and pickle df
# df1 = pd.read_csv('C:/Users/ping/OneDrive/Documents/jenn_bb_sales/items-2021-02-01-2021-10-02.csv')
# df2 = pd.read_csv('C:/Users/ping/OneDrive/Documents/jenn_bb_sales/items-2021-09-06-2022-09-07.csv')
# df = pd.concat([df1, df2])
# df = df.drop_duplicates()  # drop overlap between the 2 csv files
# # df.sort_values(by=['Date', 'Time'])
# # df.reset_index(drop=True)  # create new index
# df = df.sort_values(by=['Date', 'Time'])
# df = df.reset_index(drop=True)  # create new index
# pickle_dump(df, path_pickle_dump, 'df_sq_download')
# df.shape

In [ ]:
# df = pickle_load(path_pickle_dump, 'df_sq_download')
# df_shape_before_clean = df.shape
# # select only these columns
# df = df[['Date', 'Time', 'Category', 'Item', 'Qty',
#        'Price Point Name', 'Gross Sales',
#        'Discounts', 'Net Sales', 'Tax', 
#        'Device Name', 'Notes', 'Event Type', 
#        'Dining Option', 'Customer Name']]
# # strip leading $, convert from object to float
# cols_to_clean = ['Gross Sales', 'Discounts', 'Net Sales', 'Tax']
# for col in cols_to_clean:
#   df[col] = df[col].str.split('$').str[-1]  # strip leading $
#   df[col] = pd.to_numeric(df[col])  # convert from object to float
# df_shape_after_clean = df.shape
# pickle_dump(df, path_pickle_dump, 'df_clean')
# print(f'df.shape before clean:  {df_shape_before_clean}')
# print(f'df.shape after clean:   {df_shape_after_clean}')

In [2]:
df = pickle_load(path_pickle_dump, 'df_clean')
print(f'df.shape: {df.shape}')

df.shape: (18366, 15)


In [4]:
data_1 = df.groupby('Date')['Gross Sales'].sum()

In [6]:
data_1
print(type(data_1))

<class 'pandas.core.series.Series'>


In [13]:
df_gross = pd.DataFrame(data_1)
df_gross

,Gross Sales
Date,
2021-07-08,30.0
2021-07-09,859.0
2021-07-10,1205.0
2021-07-16,847.0
2021-07-17,1026.0
...,...
2022-09-01,64.0
2022-09-02,400.0
2022-09-03,589.0


In [24]:
df_gross.index = pd.to_datetime(df_gross.index)
df_gross.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 187 entries, 2021-07-08 to 2022-09-06
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Gross Sales  187 non-null    float64
dtypes: float64(1)
memory usage: 2.9 KB


In [10]:
df_temp = pd.read_csv('C:/Users/ping/OneDrive/Documents/jenn_bb_sales/la_temp.csv')

In [ ]:
df_temp = df_temp.set_index("Date")
df_temp

In [25]:
df_temp.index = pd.to_datetime(df_temp.index)
df_temp

,Prcp,Tmax,DoW
Date,,,
2021-01-01,0.0,70,Fri
2021-01-02,0.0,66,Sat
2021-01-03,0.0,64,Sun
2021-01-04,0.0,62,Mon
2021-01-05,0.0,63,Tue
...,...,...,...
2022-08-30,0.0,84,Tue
2022-08-31,0.0,91,Wed
2022-09-01,0.0,96,Thu


In [27]:
df_temp.columns

Index(['Prcp', 'Tmax', 'DoW'], dtype='object')

In [32]:
df_gross_temp = pd.concat([df_gross, df_temp], axis=1, join='inner')

In [35]:
pickle_dump(df_gross_temp, path_pickle_dump, 'df_gross_temp')
df_gross_temp = pickle_load(path_pickle_dump, 'df_gross_temp')
df_gross_temp

,Gross Sales,Prcp,Tmax,DoW
Date,,,,
2021-07-08,30.0,0.0,84,Thu
2021-07-09,859.0,0.0,86,Fri
2021-07-10,1205.0,0.0,85,Sat
2021-07-16,847.0,0.0,82,Fri
2021-07-17,1026.0,0.0,85,Sat
...,...,...,...,...
2022-08-20,768.0,0.0,82,Sat
2022-08-21,935.5,0.0,80,Sun
2022-09-01,64.0,0.0,96,Thu
